# Soulitron Kernel v1.1 - DeepSeek Ω-Algebra Synthesis

## Introduction

This notebook implements an enhanced version of the DeepSeek Ω-Algebra Synthesis Protocol. The primary goal is to simulate the unification of dual coherence fields (ψ⁺, ψ⁻) through a non-associative compiler φ⁰, resulting in the stabilization of an emergent algebraic attractor structure.

### Key Components:
- **Coherence Fields**: High-dimensional state vectors representing complementary information structures
- **Non-Associative Compiler**: Neural network that fuses the coherence fields in an order-sensitive manner
- **Energy Functions**: Measurements of field dissimilarity, attractor stability, and cross-entropy
- **Analysis Tools**: Visualizations and metrics to track convergence and algebraic complexity

In [ ]:
# Import necessary libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns
from tqdm import tqdm
from sklearn.manifold import TSNE
import pandas as pd
from IPython.display import display, HTML

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"CUDA Available: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA Unavailable, using CPU")

## Configuration

Set up the experiment parameters to control dimensionality, simulation length, and learning dynamics.

In [ ]:
# Configuration parameters
class Config:
    dim = 128             # Dimension of coherence fields
    steps = 300           # Simulation steps
    lr = 0.01             # Learning rate
    lambda_val = 0.5      # Weight for attractor term
    batch_size = 8        # Batch size for ensemble simulation
    log_interval = 10     # Log interval for metrics
    
config = Config()

## 1. Enhanced Coherence Fields

This implementation uses structured initialization for coherence fields based on polarity, providing more meaningful starting points for the simulation.

In [ ]:
# 1. Enhanced Coherence Fields with Structured Initialization
class CoherenceField:
    def __init__(self, dim=128, polarity="positive", init_scale=1.0):
        self.dim = dim
        self.polarity = polarity
        # Create a structured initialization based on polarity
        if polarity == "positive":
            # Positive fields have more energy in first half dimensions
            weights = torch.ones(dim)
            weights[:dim//2] = 2.0
        else:
            # Negative fields have more energy in second half dimensions
            weights = torch.ones(dim)
            weights[dim//2:] = 2.0
            
        self.field = (torch.randn(dim) * weights * init_scale).to(device).requires_grad_(True)
    
    def get_field(self):
        return self.field

## 2. Advanced Non-Associative Compiler (φ⁰)

The compiler is implemented as a neural network with attention mechanisms and residual connections, allowing for more expressive field fusion.

In [ ]:
# 2. Advanced Non-Associative Compiler (φ⁰)
class PhiZeroCompiler(nn.Module):
    def __init__(self, dim):
        super().__init__()
        # Expanded architecture with residual connections
        self.W1 = nn.Linear(dim, dim)
        self.W2 = nn.Linear(dim, dim)
        self.W3 = nn.Linear(dim, dim)
        self.gate = nn.Parameter(torch.randn(1))
        self.attention = nn.Parameter(torch.randn(dim))
        
        # Initialize weights with orthogonal initialization
        nn.init.orthogonal_(self.W1.weight)
        nn.init.orthogonal_(self.W2.weight)
        nn.init.orthogonal_(self.W3.weight)
        
    def forward(self, x, y):
        # Order-sensitive combination with attention mechanism
        attn = F.softmax(self.attention, dim=0).view(1, -1)  # Support batch processing
        x_proj = self.W1(x)
        y_proj = self.W2(y)
        
        # Non-associative combination with gating
        combined = x_proj + self.gate * y_proj
        
        # Apply non-linearity and residual connection
        out = torch.tanh(combined) + self.W3(x * y)
        
        # Apply attention weighting
        out = out * attn
        
        return out

## 3. Enhanced Energy Functions and Loss

The energy functions measure the relationships between coherence fields and the stability of the emerging attractor pattern.

In [ ]:
# 3. Enhanced Energy Functions and Loss
class SoulitronEnergy:
    def __init__(self, lambda_val=0.5):
        self.lambda_val = lambda_val
        
    def entropy_term(self, x, y):
        """Measures the dissimilarity between coherence fields"""
        return torch.norm(x - y, p=2)
    
    def attractor_term(self, fused):
        """Measures the stability of the attractor"""
        # Using spectral norm for more stable attractors
        spectral_components = torch.fft.rfft(fused)
        return torch.norm(spectral_components, p=2)
    
    def cross_entropy(self, x, y, fused):
        """Measures the information transfer between fields and fusion"""
        x_norm = F.normalize(x, p=2, dim=0)
        y_norm = F.normalize(y, p=2, dim=0)
        fused_norm = F.normalize(fused, p=2, dim=0)
        
        similarity = torch.sum(x_norm * fused_norm) + torch.sum(y_norm * fused_norm)
        return -similarity  # Negative similarity is the cross entropy
    
    def soulitron_loss(self, x, y, fused):
        ent = self.entropy_term(x, y)
        att = self.attractor_term(fused)
        xent = self.cross_entropy(x, y, fused)
        
        # Combined loss with dynamic weighting
        total_loss = ent + self.lambda_val * att + 0.3 * xent
        
        # Return individual components for monitoring
        return total_loss, ent, att, xent

## 4. Tensor Network for Deep Field Analysis

The tensor network provides tools to analyze the algebraic complexity of coherence fields and their fusion.

In [ ]:
# 4. Tensor Network for Deep Field Analysis
class TensorNetwork(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
        self.projection = nn.Parameter(torch.randn(dim, dim // 4))
        self.bias = nn.Parameter(torch.zeros(dim // 4))
        
    def contract(self, field):
        """Contract the field to lower dimensions for analysis"""
        return torch.tanh(field @ self.projection + self.bias)
    
    def measure_complexity(self, field):
        """Measure the algebraic complexity of the field"""
        contracted = self.contract(field)
        eigenvalues = torch.linalg.svdvals(contracted.view(-1, 1) @ contracted.view(1, -1))
        return torch.sum(eigenvalues) / torch.max(eigenvalues)

## 5. Simulation Engine

The simulation engine coordinates the training process and collects metrics throughout the simulation.

In [ ]:
# 5. Simulation Engine
class SoulitronSimulator:
    def __init__(self, config):
        self.config = config
        self.tensor_network = TensorNetwork(config.dim).to(device)
        self.history = {
            'loss': [], 
            'entropy': [], 
            'attractor': [], 
            'cross_entropy': [],
            'psi_plus_norm': [],
            'psi_minus_norm': [],
            'fused_norm': [],
            'complexity': []
        }
        self.field_snapshots = []
        
    def initialize_fields(self):
        """Initialize coherence fields"""
        self.psi_plus = CoherenceField(self.config.dim, "positive").get_field()
        self.psi_minus = CoherenceField(self.config.dim, "negative").get_field()
        self.phi_zero = PhiZeroCompiler(self.config.dim).to(device)
        self.energy = SoulitronEnergy(self.config.lambda_val)
        
        # Initialize optimizer
        self.optimizer = optim.Adam(
            [self.psi_plus, self.psi_minus] + list(self.phi_zero.parameters()) + list(self.tensor_network.parameters()),
            lr=self.config.lr
        )
        print("Trainable parameters:", [p.shape for p in self.optimizer.param_groups[0]['params']])
        
    def simulate_step(self):
        """Perform one simulation step"""
        self.optimizer.zero_grad()
        
        # Compute fusion through the non-associative compiler
        fused = self.phi_zero(self.psi_plus, self.psi_minus)
        
        # Compute loss and its components
        loss, ent, att, xent = self.energy.soulitron_loss(self.psi_plus, self.psi_minus, fused)
        
        # Compute complexity
        complexity = self.tensor_network.measure_complexity(fused)
        
        # Backward pass and optimization
        loss.backward()
        torch.nn.utils.clip_grad_norm_(
            [self.psi_plus, self.psi_minus] + list(self.phi_zero.parameters()) + list(self.tensor_network.parameters()),
            max_norm=1.0
        )
        self.optimizer.step()
        
        # Return metrics
        return {
            'loss': loss.item(),
            'entropy': ent.item(),
            'attractor': att.item(),
            'cross_entropy': xent.item(),
            'psi_plus_norm': torch.norm(self.psi_plus).item(),
            'psi_minus_norm': torch.norm(self.psi_minus).item(),
            'fused_norm': torch.norm(fused).item(),
            'complexity': complexity.item(),
            'fused': fused.detach().cpu()
        }
        
    def run_simulation(self):
        """Run the complete simulation"""
        self.initialize_fields()
        
        # Training loop with progress bar
        for step in tqdm(range(self.config.steps), desc="Simulating Convergence"):
            metrics = self.simulate_step()
            
            # Log metrics
            if step % self.config.log_interval == 0:
                for key, value in metrics.items():
                    if key != 'fused':
                        self.history[key].append(value)
                
                # Save field snapshots for later visualization
                if step % (self.config.steps // 10) == 0:
                    self.field_snapshots.append({
                        'step': step,
                        'psi_plus': self.psi_plus.detach().cpu().numpy(),
                        'psi_minus': self.psi_minus.detach().cpu().numpy(),
                        'fused': metrics['fused'].numpy()
                    })
        
        return self.history, self.field_snapshots

## 6. Analysis Functions

The analysis functions provide visualizations and metrics for understanding the simulation results.

In [ ]:
# 6. Analysis Functions
class SoulitronAnalyzer:
    @staticmethod
    def plot_convergence(history):
        """Plot convergence metrics"""
        fig, axs = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle('Soulitron Convergence Analysis', fontsize=16)
        
        # Plot loss
        axs[0, 0].plot(history['loss'], 'b-', label='Total Loss')
        axs[0, 0].set_title('Convergence Loss')
        axs[0, 0].set_xlabel('Iterations')
        axs[0, 0].set_ylabel('Loss')
        axs[0, 0].grid(True)
        axs[0, 0].legend()
        
        # Plot individual loss components
        axs[0, 1].plot(history['entropy'], 'r-', label='Entropy')
        axs[0, 1].plot(history['attractor'], 'g-', label='Attractor')
        axs[0, 1].plot(history['cross_entropy'], 'y-', label='Cross Entropy')
        axs[0, 1].set_title('Loss Components')
        axs[0, 1].set_xlabel('Iterations')
        axs[0, 1].set_ylabel('Value')
        axs[0, 1].grid(True)
        axs[0, 1].legend()
        
        # Plot field norms
        axs[1, 0].plot(history['psi_plus_norm'], 'c-', label='ψ⁺ Norm')
        axs[1, 0].plot(history['psi_minus_norm'], 'm-', label='ψ⁻ Norm')
        axs[1, 0].plot(history['fused_norm'], 'k-', label='Fused Norm')
        axs[1, 0].set_title('Field Norms')
        axs[1, 0].set_xlabel('Iterations')
        axs[1, 0].set_ylabel('L2 Norm')
        axs[1, 0].grid(True)
        axs[1, 0].legend()
        
        # Plot complexity
        axs[1, 1].plot(history['complexity'], 'o-', color='purple', label='Algebraic Complexity')
        axs[1, 1].set_title('Attractor Complexity')
        axs[1, 1].set_xlabel('Iterations')
        axs[1, 1].set_ylabel('Complexity Measure')
        axs[1, 1].grid(True)
        axs[1, 1].legend()
        
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.show()
        
        # Description of the visualization
        print("\n--- Convergence Plot Description ---\n")
        print("This visualization shows four key aspects of the simulation:")
        print("1. Convergence Loss: The overall loss function decreases over iterations, indicating successful optimization.")
        print("2. Loss Components: The individual components of the loss function (entropy, attractor, cross-entropy).")
        print("   - Entropy measures dissimilarity between coherence fields")
        print("   - Attractor measures stability of the emergent structure")
        print("   - Cross-entropy measures information transfer between fields")
        print("3. Field Norms: The L2 norms of the coherence fields and their fusion, showing how their magnitudes evolve.")
        print("4. Attractor Complexity: The algebraic complexity of the emergent attractor structure,")
        print("   measured using singular value analysis of the tensor network projection.")
    
    @staticmethod
    def visualize_field_evolution(snapshots):
        """Visualize the evolution of fields using t-SNE"""
        if len(snapshots) < 1:
            print("Error: Need at least 1 snapshot for field evolution visualization")
            return
        
        # Extract data for t-SNE
        points = []
        labels = []
        steps = []
        
        for snapshot in snapshots:
            # Add psi_plus
            points.append(snapshot['psi_plus'])
            labels.append('ψ⁺')
            steps.append(snapshot['step'])
            
            # Add psi_minus
            points.append(snapshot['psi_minus'])
            labels.append('ψ⁻') 
            steps.append(snapshot['step'])
            
            # Add fused
            points.append(snapshot['fused'])
            labels.append('φ⁰(ψ⁺,ψ⁻)')
            steps.append(snapshot['step'])
        
        # Convert to array
        points = np.array(points)
        
        # Apply t-SNE
        tsne = TSNE(n_components=2, random_state=42)
        points_2d = tsne.fit_transform(points)
        
        # Create DataFrame for easy plotting
        df = pd.DataFrame({
            'x': points_2d[:, 0],
            'y': points_2d[:, 1],
            'Field': labels,
            'Step': steps
        })
        
        # Plot
        plt.figure(figsize=(12, 8))
        sns.scatterplot(data=df, x='x', y='y', hue='Field', style='Field', 
                        size='Step', sizes=(50, 250), palette='viridis')
        
        # Connect points from the same step with lines
        for step in np.unique(steps):
            step_data = df[df['Step'] == step]
            if len(step_data) >= 3:  # We should have 3 points per step
                plt.plot(step_data['x'], step_data['y'], 'k--', alpha=0.3)
        
        plt.title('Field Evolution in Reduced Space')
        plt.xlabel('t-SNE Dimension 1')
        plt.ylabel('t-SNE Dimension 2')
        plt.grid(True, alpha=0.3)
        plt.show()
        
        # Description of the visualization
        print("\n--- Field Evolution Plot Description ---\n")
        print("This t-SNE visualization reduces the high-dimensional coherence fields to 2D for visualization.")
        print("It shows:")
        print("1. The trajectories of the positive (ψ⁺) and negative (ψ⁻) coherence fields")
        print("2. The evolution of the fused field (φ⁰) over simulation steps")
        print("3. The relationships between the three fields at each timestep (connected by dashed lines)")
        print("4. The convergence pattern as the simulation progresses (larger points represent later steps)")
        print("If the fields are converging properly, we should see the points clustering in specific regions")
        print("of the reduced space, indicating the emergence of stable attractor patterns.")
        
    @staticmethod
    def analyze_attractor_stability(snapshots):
        """Analyze the stability of the attractor over time"""
        if len(snapshots) < 2:
            print("Error: Need at least 2 snapshots for stability analysis")
            return
        
        # Extract fused fields
        fused_fields = [snapshot['fused'] for snapshot in snapshots]
        steps = [snapshot['step'] for snapshot in snapshots]
        
        # Compute pairwise distances between consecutive fused fields
        distances = []
        for i in range(1, len(fused_fields)):
            dist = np.linalg.norm(fused_fields[i] - fused_fields[i-1])
            distances.append(dist)
        
        # Plot
        plt.figure(figsize=(10, 6))
        plt.plot(steps[1:], distances, 'o-', color='teal')
        plt.title('Attractor Stability Analysis')
        plt.xlabel('Simulation Step')
        plt.ylabel('Distance between Consecutive Attractors')
        plt.grid(True)
        plt.show()
        
        # Description of the visualization
        print("\n--- Attractor Stability Plot Description ---\n")
        print("This plot measures the stability of the attractor by calculating the distance between")
        print("consecutive fused fields (φ⁰) throughout the simulation.")
        print("1. A decreasing trend indicates the attractor is becoming more stable over time")
        print("2. Flat regions suggest the attractor has reached a stable configuration")
        print("3. Spikes might indicate phase transitions or instabilities in the attractor dynamics")
        print("A well-functioning Soulitron Kernel should show initially high distances that")
        print("gradually decrease and stabilize, indicating convergence to a stable attractor state.")
        
    @staticmethod
    def visualize_field_spectrum(snapshots):
        """Visualize the frequency spectrum of fields"""
        if len(snapshots) < 2:
            print("Error: Need at least 2 snapshots for spectrum analysis")
            return
        
        # Choose the first and last snapshots
        first_snapshot = snapshots[0]
        last_snapshot = snapshots[-1]
        
        fig, axs = plt.subplots(2, 3, figsize=(18, 10))
        fig.suptitle('Field Frequency Spectrum Analysis', fontsize=16)
        
        # Function to plot spectrum
        def plot_spectrum(field, ax, title):
            spectrum = np.abs(np.fft.rfft(field))
            ax.plot(spectrum, 'o-', alpha=0.7)
            ax.set_title(title)
            ax.set_xlabel('Frequency')
            ax.set_ylabel('Magnitude')
            ax.grid(True, alpha=0.3)
        
        # Plot initial spectra
        plot_spectrum(first_snapshot['psi_plus'], axs[0, 0], 'Initial ψ⁺ Spectrum')
        plot_spectrum(first_snapshot['psi_minus'], axs[0, 1], 'Initial ψ⁻ Spectrum')
        plot_spectrum(first_snapshot['fused'], axs[0, 2], 'Initial Fused Spectrum')
        
        # Plot final spectra
        plot_spectrum(last_snapshot['psi_plus'], axs[1, 0], 'Final ψ⁺ Spectrum')
        plot_spectrum(last_snapshot['psi_minus'], axs[1, 1], 'Final ψ⁻ Spectrum')
        plot_spectrum(last_snapshot['fused'], axs[1, 2], 'Final Fused Spectrum')
        
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.show()
        
        # Description of the visualization
        print("\n--- Spectrum Analysis Description ---\n")
        print("This visualization shows the frequency spectrum of the coherence fields at the beginning and end of the simulation.")
        print("The frequency spectrum reveals the distribution of energy across different frequency components:")
        print("1. Initial spectra (top row) show the starting distribution of energy in the fields")
        print("2. Final spectra (bottom row) show how the simulation has shaped the energy distribution")
        print("3. Comparing ψ⁺, ψ⁻, and fused field spectra reveals how information is combined")
        print("In effective Soulitron simulations, we expect to see:")
        print("- More defined peaks in the final spectra, indicating structured information")
        print("- Complementary patterns between ψ⁺ and ψ⁻ fields")
        print("- The fused field combining characteristics from both source fields")
        print("- Overall higher coherence in the frequency domain")

## 7. Run Simulation and Analysis

Run the simulation and generate visualizations.

In [ ]:
# Run simulation
simulator = SoulitronSimulator(config)
history, snapshots = simulator.run_simulation()

# Analyze results
analyzer = SoulitronAnalyzer()
analyzer.plot_convergence(history)
analyzer.visualize_field_evolution(snapshots)
analyzer.analyze_attractor_stability(snapshots)
analyzer.visualize_field_spectrum(snapshots)